In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [5]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-CIVDEF/DRM-CIVDEF.csv", encoding="cp949")
print("원본 크기:", df.shape)
display(df.head())

# 2. 주소에서 자치구 추출
df["gu"] = df["도로명주소"].str.extract(r"서울특별시\s*([가-힣]+구)")
# df["gu"] = df["gu"].str.replace("구$","",regex=True)

# 3. 사용중(정상 운영)만 필터링
df = df[df["영업상태명"] == "영업/정상"].copy()

# 4. 소재지면적 전처리
df["소재지면적"] = pd.to_numeric(df["소재지면적"], errors="coerce").fillna(0)

# 5. 구별 집계
agg = df.groupby("gu").agg(
    civdef_count=("관리번호", "count"),
    civdef_total_area=("소재지면적", "sum"),
    civdef_avg_area=("소재지면적", "mean")
).reset_index()

display(agg.head())

# 6. 정규화 (많을수록 안전에 유리 → 정방향 점수)
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["civdef_count","civdef_total_area","civdef_avg_area"]])

score_df = agg[["gu"]].copy()
for i, col in enumerate(["civdef_count","civdef_total_area","civdef_avg_area"]):
    score_df[col+"_score"] = scaled[:, i]

# 7. 종합 점수 (평균)
score_df["civdef_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 8. 저장
score_df.to_csv("../../result/DRM/DRM-CIVDEF_result.csv", index=False, encoding="utf-8-sig")

score_df.head()

원본 크기: (2736, 29)


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),비상시설위치,시설구분명,시설명_건물명,해제일자
0,3080000,3080000-S202300006,2023-07-31,NaN,1,영업/정상,18,사용중,NaN,NaN,...,2024-01-06 20:08:45,U,2023-12-01 00:08:00.0,NaN,201045.611312,457292.961457,NaN,NaN,NaN,NaN
1,3080000,3080000-S202400001,2024-01-22,NaN,1,영업/정상,18,사용중,NaN,NaN,...,2024-01-30 12:33:59,U,2023-12-02 00:01:00.0,NaN,202624.617263,456818.852071,NaN,NaN,NaN,NaN
2,3030000,3030000-S202300027,2023-07-07,NaN,1,영업/정상,18,사용중,NaN,NaN,...,2023-07-18 13:44:25,I,2022-12-06 22:00:00.0,NaN,204522.096092,448925.421927,NaN,NaN,NaN,NaN
3,3160000,3160000-S202200004,2022-12-22,NaN,1,영업/정상,18,사용중,NaN,NaN,...,2024-02-02 09:45:24,U,2023-12-02 00:04:00.0,NaN,190173.233224,442268.896000,NaN,NaN,NaN,NaN
4,3100000,3100000-S201700003,2017-05-23,NaN,1,영업/정상,18,사용중,NaN,NaN,...,2024-02-02 14:27:18,U,2023-12-02 00:04:00.0,NaN,205198.498721,457486.206509,NaN,NaN,NaN,NaN


,gu,civdef_count,civdef_total_area,civdef_avg_area
0,강남구,76,526629.87,6929.340395
1,강동구,60,1107266.62,18454.443667
2,강북구,87,487243.44,5600.499310
3,강서구,167,1940246.81,11618.244371
4,관악구,78,500584.84,6417.754359


,gu,civdef_count_score,civdef_total_area_score,civdef_avg_area_score,civdef_mean_score
0,강남구,20.175439,11.560644,13.813908,15.183330
1,강동구,6.140351,41.376503,88.324309,45.280388
2,강북구,29.824561,9.538139,5.222880,14.861860
3,강서구,100.000000,84.150269,44.127920,76.092730
4,관악구,21.929825,10.223224,10.506477,14.219842


gu (자치구명)

civdef_count_score (민방위시설 개수 점수)

civdef_total_area_score (총 면적 점수)

civdef_avg_area_score (평균 면적 점수)

civdef_mean_score (평균 종합 점수)